---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-machine-learning/resources/bANLa) course resource._

---

## Assignment 4 - Understanding and Predicting Property Maintenance Fines

This assignment is based on a data challenge from the Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)). 

The Michigan Data Science Team ([MDST](http://midas.umich.edu/mdst/)) and the Michigan Student Symposium for Interdisciplinary Statistical Sciences ([MSSISS](https://sites.lsa.umich.edu/mssiss/)) have partnered with the City of Detroit to help solve one of the most pressing problems facing Detroit - blight. [Blight violations](http://www.detroitmi.gov/How-Do-I/Report/Blight-Complaint-FAQs) are issued by the city to individuals who allow their properties to remain in a deteriorated condition. Every year, the city of Detroit issues millions of dollars in fines to residents and every year, many of these fines remain unpaid. Enforcing unpaid blight fines is a costly and tedious process, so the city wants to know: how can we increase blight ticket compliance?

The first step in answering this question is understanding when and why a resident might fail to comply with a blight ticket. This is where predictive modeling comes in. For this assignment, your task is to predict whether a given blight ticket will be paid on time.

All data for this assignment has been provided to us through the [Detroit Open Data Portal](https://data.detroitmi.gov/). **Only the data already included in your Coursera directory can be used for training the model for this assignment.** Nonetheless, we encourage you to look into data from other Detroit datasets to help inform feature creation and model selection. We recommend taking a look at the following related datasets:

* [Building Permits](https://data.detroitmi.gov/Property-Parcels/Building-Permits/xw2a-a7tf)
* [Trades Permits](https://data.detroitmi.gov/Property-Parcels/Trades-Permits/635b-dsgv)
* [Improve Detroit: Submitted Issues](https://data.detroitmi.gov/Government/Improve-Detroit-Submitted-Issues/fwz3-w3yn)
* [DPD: Citizen Complaints](https://data.detroitmi.gov/Public-Safety/DPD-Citizen-Complaints-2016/kahe-efs3)
* [Parcel Map](https://data.detroitmi.gov/Property-Parcels/Parcel-Map/fxkw-udwf)

___

We provide you with two data files for use in training and validating your models: train.csv and test.csv. Each row in these two files corresponds to a single blight ticket, and includes information about when, why, and to whom each ticket was issued. The target variable is compliance, which is True if the ticket was paid early, on time, or within one month of the hearing data, False if the ticket was paid after the hearing date or not at all, and Null if the violator was found not responsible. Compliance, as well as a handful of other variables that will not be available at test-time, are only included in train.csv.

Note: All tickets where the violators were found not responsible are not considered during evaluation. They are included in the training set as an additional source of data for visualization, and to enable unsupervised and semi-supervised approaches. However, they are not included in the test set.

<br>

**File descriptions** (Use only this data for training your model!)

    train.csv - the training set (all tickets issued 2004-2011)
    test.csv - the test set (all tickets issued 2012-2016)
    addresses.csv & latlons.csv - mapping from ticket id to addresses, and from addresses to lat/lon coordinates. 
     Note: misspelled addresses may be incorrectly geolocated.

<br>

**Data fields**

train.csv & test.csv

    ticket_id - unique identifier for tickets
    agency_name - Agency that issued the ticket
    inspector_name - Name of inspector that issued the ticket
    violator_name - Name of the person/organization that the ticket was issued to
    violation_street_number, violation_street_name, violation_zip_code - Address where the violation occurred
    mailing_address_str_number, mailing_address_str_name, city, state, zip_code, non_us_str_code, country - Mailing address of the violator
    ticket_issued_date - Date and time the ticket was issued
    hearing_date - Date and time the violator's hearing was scheduled
    violation_code, violation_description - Type of violation
    disposition - Judgment and judgement type
    fine_amount - Violation fine amount, excluding fees
    admin_fee - $20 fee assigned to responsible judgments
state_fee - $10 fee assigned to responsible judgments
    late_fee - 10% fee assigned to responsible judgments
    discount_amount - discount applied, if any
    clean_up_cost - DPW clean-up or graffiti removal cost
    judgment_amount - Sum of all fines and fees
    grafitti_status - Flag for graffiti violations
    
train.csv only

    payment_amount - Amount paid, if any
    payment_date - Date payment was made, if it was received
    payment_status - Current payment status as of Feb 1 2017
    balance_due - Fines and fees still owed
    collection_status - Flag for payments in collections
    compliance [target variable for prediction] 
     Null = Not responsible
     0 = Responsible, non-compliant
     1 = Responsible, compliant
    compliance_detail - More information on why each ticket was marked compliant or non-compliant


___

## Evaluation

Your predictions will be given as the probability that the corresponding blight ticket will be paid on time.

The evaluation metric for this assignment is the Area Under the ROC Curve (AUC). 

Your grade will be based on the AUC score computed for your classifier. A model which with an AUROC of 0.7 passes this assignment, over 0.75 will recieve full points.
___

For this assignment, create a function that trains a model to predict blight ticket compliance in Detroit using `train.csv`. Using this model, return a series of length 61001 with the data being the probability that each corresponding ticket from `test.csv` will be paid, and the index being the ticket_id.

Example:

    ticket_id
       284932    0.531842
       285362    0.401958
       285361    0.105928
       285338    0.018572
                 ...
       376499    0.208567
       376500    0.818759
       369851    0.018528
       Name: compliance, dtype: float32

In [91]:
import pandas as pd
from numpy import *
#from pylab import *
#import seaborn as sns
#%matplotlib inline
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import KFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier


def blight_model():
    #load test/train data
    train = pd.read_csv('train.csv',encoding = 'ISO-8859-1')
    test = pd.read_csv('test.csv')
    
    #replace nan
    train = train.fillna('NaN')
    test = test.fillna('NaN')
    
    #drop columns that have uninformative/bad values
    cols2drop = ['violation_zip_code','non_us_str_code',\
                 'grafitti_status','ticket_issued_date','hearing_date',\
                 'zip_code','mailing_address_str_number','violator_name',\
                'country','clean_up_cost']

    for c in cols2drop:
        train.drop(c,axis=1,inplace=1)
        test.drop(c,axis=1,inplace=1)    
    fitcols=test.columns

    '''
    #couldn't get this to work on coursera server, so dropped it
    train['ticket_issued_date'] = pd.to_datetime(train.ticket_issued_date,format='%Y-%m-%d %H:%M:%S').astype('int64')/1e9
    test['ticket_issued_date'] = pd.to_datetime(test.ticket_issued_date,format='%Y-%m-%d %H:%M:%S').astype('int64')/1e9
    train['hearing_date'] = pd.to_datetime(train.hearing_date,format='%Y-%m-%d %H:%M:%S').astype('int64')/1e9
    test['hearing_date'] = pd.to_datetime(test.hearing_date,format='%Y-%m-%d %H:%M:%S').astype('int64')/1e9
    '''

    #label categories as int's
    number = preprocessing.LabelEncoder()
    paystat = pd.Series(number.fit_transform(train['payment_status'].astype('str')))
    paystat2 = paystat.tolist()
    for i,j in enumerate(paystat2):
        if j==2:
            paystat2[i]=1
    paystat2 = pd.Series(paystat2)
    
    #join all data together so I can correctly encode the values
    train['label'] = 'train'
    test['label'] = 'test'
    total = pd.concat([train,test])
        
    #cols2onehot = []
    cols2encode = ['agency_name','inspector_name',\
                   'violation_street_name',\
                  'mailing_address_str_name','city','state',\
                   'violation_description','disposition',\
                   'violation_code'] #'country',\
    '''
    for c in cols2onehot:
        print (c)
        tdf = pd.get_dummies(total[c],drop_first=1,prefix=c).astype(int)
        total = pd.concat([total,tdf],axis=1)
        total.drop(c,inplace=1,axis=1)
        '''
    for c in cols2encode:
        #print (c)
        total[c] = number.fit_transform(total[c])
    
    #split the columns back
    ctest = total[total.label =='test']
    ctest = ctest[fitcols]#ctest.drop('label',axis=1,inplace=1)

    ctrain = total[total.label =='train']
    ctrain = ctrain[fitcols]
    
    #make nan=-1
    ctrain = ctrain.fillna(-1)
    ctest = ctest.fillna(-1)

    #choose features and target
    X = ctrain
    y = paystat2
    X_train,X_test,y_train,y_test = train_test_split(X,y,random_state=0)

    #use random forest classifier
    rf = RandomForestClassifier(random_state=0,n_estimators=20,max_depth=5)
    rf.fit(X_train, y_train)
    #print (sorted(zip(map(lambda x: round(x, 4), rf.feature_importances_),\
    #                  X.columns), reverse=True))
    
    #predict probability of ticket being paid
    prob = rf.predict_proba(ctest)[:,1]
    pdf = pd.Series(prob,index=ctest.ticket_id)
  
    return pdf

In [92]:
blight_model()

/opt/conda/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2827: DtypeWarning: Columns (11,12,31) have mixed types. Specify dtype option on import or set low_memory=False.
  if self.run_code(code, result):


ticket_id
284932    0.226908
285362    0.142433
285361    0.233322
285338    0.213071
285346    0.233360
285345    0.213071
285347    0.245273
285342    0.727862
285530    0.157787
284989    0.142433
285344    0.213393
285343    0.145165
285340    0.141575
285341    0.213393
285349    0.242459
285348    0.228134
284991    0.142433
285532    0.142433
285406    0.142433
285001    0.138426
285006    0.142804
285405    0.141575
285337    0.157787
285496    0.214252
285497    0.213930
285378    0.142433
285589    0.157787
285585    0.213930
285501    0.233322
285581    0.142433
            ...   
376367    0.142804
376366    0.151238
376362    0.308613
376363    0.356693
376365    0.142804
376364    0.151238
376228    0.149056
376265    0.152344
376286    0.917162
376320    0.151238
376314    0.151238
376327    0.917162
376385    0.917162
376435    0.696210
376370    0.866912
376434    0.245273
376459    0.219485
376478    0.145062
376473    0.149056
376484    0.114470
376482    0.109770
37